### This is the first part of @afajohn kernel with the addition of multiprocessing on the test set
Check out his brilliant kernel here https://www.kaggle.com/afajohn/cnn-lstm-for-signal-classification-lb-0-513

Also, many thanks to following kernels:
- For shortening the signals with a simple feature extraction thanks to: https://www.kaggle.com/ashishpatel26/transfer-learning-in-basic-nn
- For signal denoising and fft: https://www.kaggle.com/theoviel/fast-fourier-transform-denoising

In [1]:
import sys
import gc
import time
import os
import logging
from multiprocessing import Pool, current_process
from multiprocessing import log_to_stderr, get_logger
from tqdm import tqdm
from numba import jit

import pyarrow.parquet as pq
import pandas as pd
import numpy as np

import keras
import keras.backend as K
from keras.layers import LSTM,Dropout,Dense,TimeDistributed,Conv1D,MaxPooling1D,Flatten
from keras.models import Sequential
import tensorflow as tf

from IPython.display import display, clear_output

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

sns.set_style("whitegrid")

Using TensorFlow backend.


In [2]:
import pyarrow.parquet as pq
import pandas as pd
import numpy as np

In [3]:
%%time 
train_set = pq.read_pandas('../input/train.parquet').to_pandas()

CPU times: user 56.5 s, sys: 25.6 s, total: 1min 22s
Wall time: 1min 48s


In [4]:
%%time
meta_train = pd.read_csv('../input/metadata_train.csv')

CPU times: user 25 ms, sys: 5.52 ms, total: 30.5 ms
Wall time: 46.6 ms


In [5]:
@jit('float32(float32[:,:], int32)')
def feature_extractor(x, n_part=1000):
    lenght = len(x)
    pool = np.int32(np.ceil(lenght/n_part))
    output = np.zeros((n_part,))
    for j, i in enumerate(range(0,lenght, pool)):
        if i+pool < lenght:
            k = x[i:i+pool]
        else:
            k = x[i:]
        output[j] = np.max(k, axis=0) - np.min(k, axis=0)
    return output

In [6]:
x_train = []
y_train = []
for i in tqdm(meta_train.signal_id):
    idx = meta_train.loc[meta_train.signal_id==i, 'signal_id'].values.tolist()
    y_train.append(meta_train.loc[meta_train.signal_id==i, 'target'].values)
    x_train.append(abs(feature_extractor(train_set.iloc[:, idx].values, n_part=400)))

100%|██████████| 8712/8712 [01:43<00:00, 84.39it/s]


In [7]:
del train_set; gc.collect()

31671

In [8]:
y_train = np.array(y_train).reshape(-1,)
X_train = np.array(x_train).reshape(-1,x_train[0].shape[0])

In [9]:
def keras_auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc

In [10]:
n_signals = 1 #So far each instance is one signal. We will diversify them in next step
n_outputs = 1 #Binary Classification

In [11]:
#Build the model
verbose, epochs, batch_size = True, 15, 16
n_steps, n_length = 40, 10
X_train = X_train.reshape((X_train.shape[0], n_steps, n_length, n_signals))
# define model
model = Sequential()
model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu'), input_shape=(None,n_length,n_signals)))
model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu')))
model.add(TimeDistributed(Dropout(0.5)))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(100))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dense(n_outputs, activation='sigmoid'))

In [12]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[keras_auc])

In [13]:
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose)

Epoch 1/15
8712/8712 [==============================] - 29s 3ms/step - loss: 0.2217 - keras_auc: 0.5647
Epoch 2/15
8712/8712 [==============================] - 27s 3ms/step - loss: 0.1710 - keras_auc: 0.7079
Epoch 3/15
8712/8712 [==============================] - 27s 3ms/step - loss: 0.1410 - keras_auc: 0.8020
Epoch 4/15
8712/8712 [==============================] - 27s 3ms/step - loss: 0.1376 - keras_auc: 0.8464
Epoch 5/15
8712/8712 [==============================] - 28s 3ms/step - loss: 0.1249 - keras_auc: 0.8698
Epoch 6/15
8712/8712 [==============================] - 27s 3ms/step - loss: 0.1256 - keras_auc: 0.8840
Epoch 7/15
8712/8712 [==============================] - 28s 3ms/step - loss: 0.1245 - keras_auc: 0.8940
Epoch 8/15
8712/8712 [==============================] - 27s 3ms/step - loss: 0.1149 - keras_auc: 0.9018
Epoch 9/15
8712/8712 [==============================] - 27s 3ms/step - loss: 0.1153 - keras_auc: 0.9082
Epoch 10/15
8712/8712 [==============================] - 29s 3ms

In [14]:
model.save_weights('model1.hdf5')

In [15]:
#%%time
meta_test = pd.read_csv('../input/metadata_test.csv')

In [16]:
def process_chunk(arg):
    start_index = arg['start_index']
    chunk_size = arg['chunk_size']
    
    # Test set indices start at 8712
    test_set_start = 8712
    offset_index = (test_set_start + start_index)
    
    # Column name must be a string
    subset_test = pq.read_pandas('../input/test.parquet', columns=[str(offset_index + j) for j in range(chunk_size)]).to_pandas()    
    x_test = []
    for j in range(chunk_size):
        subset_test_row = subset_test[str(offset_index + j)]
        x_test.append(abs(feature_extractor(subset_test_row.values, n_part=400)))
    return x_test

In [17]:
# Define 21 chunks for processing the test set
# on multiple cpus. I have choosen to process in chunks of 1000 (plus the remainder)
# so as to keep within the kernels memory limit
args = []
for i in range(0, 20000, 1000):
    args.append({
        'start_index': i,
        'chunk_size': 1000
    })
    
# Add a chunk for the remainder
args.append({
    'start_index': 20000,
    'chunk_size': 337
})

n_cpu = processes=os.cpu_count()
print('n_cpu: ', n_cpu)

p = Pool(processes=n_cpu)

# Map the chunk args to the the process_chunk function
x_test_chunks = p.map(process_chunk, args)
print(f"multi processing complete. len: {len(x_test_chunks)}")

p.close()
p.join()

n_cpu:  4


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:14: RuntimeWarning: overflow encountered in byte_scalars
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:14: RuntimeWarning: overflow encountered in byte_scalars
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:14: RuntimeWarning: overflow encountered in byte_scalars
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:14: RuntimeWarning: overflow encountered in byte_scalars
  


multi processing complete. len: 21


In [18]:
x_test = [item for sublist in x_test_chunks for item in sublist]
x_test = np.array(x_test)
print('x_test.shape: ', x_test.shape)
X_test = x_test.reshape((x_test.shape[0], n_steps, n_length, n_signals))

x_test.shape:  (20337, 400)


In [19]:
del x_test_chunks

In [20]:
preds = model.predict(X_test)
preds.shape

(20337, 1)

In [21]:
threshpreds = (preds>0.5)*1
sub = pd.read_csv('../input/sample_submission.csv')
sub.target = threshpreds

# Gave me an LB score of ~0.450
sub.to_csv('submission.csv',index=False)

In [22]:
check_sub = pd.read_csv('submission.csv')
check_sub.head(20)

,signal_id,target
0,8712,0
1,8713,0
2,8714,0
3,8715,0
4,8716,0
5,8717,0
6,8718,0
7,8719,0
8,8720,0
9,8721,0
